In [0]:
#dbutils.widgets.text("TABLE_TRANSFORM", "g3_catalog.silver.spark_sql_transform")
#dbutils.widgets.text("TABLE_SUMMARY", "g3_catalog.gold.spark_sql_summary")

In [0]:
TABLE_TRANSFORM = dbutils.widgets.get("TABLE_TRANSFORM")
TABLE_SUMMARY = dbutils.widgets.get("TABLE_SUMMARY")

In [0]:
spark.sql(f"""
  CREATE OR REPLACE TABLE {TABLE_SUMMARY} AS
  SELECT
    claim_id,
    claim_date,
    amount,
    user_id,
    patient_name,
    birth_date,
    gender,
    enrollment_date,
    patient_status,
    diagnosis_code,
    diagnosis_description
  FROM (
    SELECT
      *,
      -- 3. Calcula el ranking porcentual de cada reclamo según su monto
      PERCENT_RANK() OVER (ORDER BY amount DESC) as amount_rank
    FROM
      {TABLE_TRANSFORM}
    -- 1. Filtra para incluir solo pacientes afiliados
    WHERE
      user_id IS NOT NULL
  )
  -- 2. Selecciona solo el 10% superior (donde el ranking es menor o igual a 0.1)
  WHERE
    amount_rank <= 0.1
""")


DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]